In [1]:
from flask import Flask, request, make_response
from flask_cors import CORS
import pandas as pd
import json
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from heapq import nsmallest
import collections 
from itertools import islice

In [2]:
# Pre-processing the data
ratings = pd.read_csv('data/animelist.csv')
anime = pd.read_csv('data/anime.csv')
anime_synopsis = pd.read_csv('data/anime_with_synopsis.csv')

anime = anime[['Name', 'MAL_ID', 'Score', 'Type']]
anime_synopsis = anime_synopsis[['Name', 'sypnopsis']]
anime = anime.rename(columns={'MAL_ID': 'anime_id'})

ratings = pd.merge(ratings, anime, on="anime_id", how="inner")
ratings = ratings[ratings['Type'] == 'TV']
ratings = ratings[['user_id', 'Name', 'rating']]

counts = ratings['user_id'].value_counts()
ratings = ratings[ratings['user_id'].isin(counts[counts >= 300].index)]




In [3]:
# Create Pivot Table
similar_animes_pivot = ratings.pivot_table(index='Name', columns='user_id', values='rating').fillna(0)
similar_animes_pivot.head()


user_id,3,12,16,17,19,21,34,41,44,46,...,353357,353358,353365,353370,353381,353383,353385,353390,353395,353398
Name,,,,,,,,,,,,,,,,,,,,,
"""Eikou Naki Tensai-tachi"" Kara no Monogatari",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Roots,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
.hack//Sign,0.0,10.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,0.0
0-sen Hayato,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Creating a sparse matrix
anime_matrix = csr_matrix(similar_animes_pivot.values)

In [5]:
# Fitting the model
anime_knn = NearestNeighbors(metric='cosine', algorithm='brute')
anime_knn.fit(anime_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [6]:
# getting synopsis
def get_synopsis(anime):
        synopsis=anime_synopsis.loc[anime_synopsis['Name'] == anime, 'sypnopsis'].item()
        return synopsis

In [7]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [8]:
# Function to get all anime's from the dataframe and process it.
def get_all_animes():
    return ratings.loc[:, ["Name"]].drop_duplicates().to_json(orient="records")

In [9]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [10]:
# Function to predict anime from the current pivot table of similar anime's based on ratings.
def generate_recommendations(anime_dict):
    recommend_dict = {}
    for index in anime_dict:
        # Measure Euclidian Distance by using Nearest Neighbours n sklearn
        distances, indices = anime_knn.kneighbors(similar_animes_pivot.loc[index, :].values.reshape(1, -1),
                                                  n_neighbors=10)
     
        # Use Euclidian Distance to find similar anime's and store them in a dictionary
        for i in range(0, len(distances.flatten())):
            if i == 0:
                continue
            else:
                dist = distances.flatten()[i] / anime_dict[index]
                index_id = similar_animes_pivot.index[indices.flatten()[i]]
                if not (index_id in anime_dict.keys() or index_id in recommend_dict.keys()):
                    recommend_dict[index_id] = dist
   
    ordered_recommend_dict = collections.OrderedDict(sorted(recommend_dict.items()))
    result = take(10, ordered_recommend_dict)
    

    return result


In [ ]:
"""
FLASK API 
"""

# Initialize empty dictionary to store the user data
user_dat_dict = {}
result_dat_dict = {}

app = Flask(__name__)
CORS(app)


@app.route('/api/animes', methods=['GET'])
def animes():
    anime_list = get_all_animes()
    return anime_list


@app.route('/api/recommendations', methods=['POST'])
def recommendations():
   
    data = request.get_json()
    for d in data:
        user_dat_dict[d["animeName"]] = int(d["rating"])
        
    user_recommendations = generate_recommendations(user_dat_dict)
    print(user_recommendations)

    for k in user_recommendations:
        
        synopsis_string = get_synopsis(k)
        result_dat_dict[k] = synopsis_string

    # Process data and return it to the input
    print(user_dat_dict)
    json_dict = []
    for recommended_anime, synopsis in result_dat_dict.items():
        json_dict.append(({'animeName': recommended_anime, 'synopsis': synopsis}))
        
        
    json_str = json.dumps(json_dict)

    
    user_dat_dict.clear()
    return make_response(json_str)
    


if __name__ == "__main__":
    app.run(port=5555, debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2022 17:48:34] "OPTIONS /api/recommendations HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 17:48:38] "POST /api/recommendations HTTP/1.1" 200 -


['Angel Beats!', 'Ao no Exorcist', 'Boku no Hero Academia', 'Boku no Hero Academia 2nd Season', 'Boku no Hero Academia 3rd Season', 'Code Geass: Hangyaku no Lelouch', 'Code Geass: Hangyaku no Lelouch R2', 'Dr. Stone', 'Dr. Stone: Stone Wars', 'Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka']
{'Ousama Ranking': 7, 'Death Note': 8, 'One Piece': 9, 'Haikyuu!!': 8, 'Haikyuu!! Second Season': 7, 'Jujutsu Kaisen (TV)': 10, 'Bleach': 7, 'Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai': 7, 'Kimetsu no Yaiba': 8}
